# SONATA Spike Train Reports

Spike-train reports are a hdf5 based format for representing neuronal spikes in a manner optimized for large-scale simulations. pySonata includes classes for creating these reports and reading them into your own code.

In [1]:
import numpy as np

from sonata.reports.spike_trains import SpikeTrains, sort_order, PoissonSpikeGenerator

## Saving Spiking information

We start by showing how to create a sonata file for storing spike trains of multi-neuron populations. 

Once we initalized a SpikeTrains object we start by adding individual spiking events. To represent a spike we need three things; the id of the node (eg cell) that spikes, the name of the population (in this case the area of the brain where the node came from), and the time of the spike. By default Sonata assumes the units are in miliseconds but that can be changed in the format.

To add an individual spike we can use the **add_spike** method

In [2]:
st_buffer = SpikeTrains()
st_buffer.add_spike(node_id=0, timestamp=1.0, population='VTA')
st_buffer.add_spike(node_id=0, timestamp=23.0, population='VTA')
st_buffer.add_spike(node_id=0, timestamp=50.1, population='VTA')


The **add_spikes** emthod allow us to save an array of spikes at a time.

In [3]:
# By setting node_ids to a single node value, all 10 spikes will be associated with node 1.
times = np.sort(np.random.uniform(0.01, 1000.0, size=10))
st_buffer.add_spikes(node_ids=1, timestamps=times, population='VTA')

In [4]:
# We can also pass in a list of nodes that is the same size as the list of spikes, creating a one-to-one correspondence
node_ids = np.random.choice([2, 3, 4, 5], size=100, replace=True)
timestamps = np.random.uniform(0.01, 1000.0, size=100)
st_buffer.add_spikes(node_ids=node_ids, timestamps=timestamps, population='VTA')

Often in simulations we are recording from cells from different populations/areas. By specifying a different population we prevent id clashes.

In [5]:
st_buffer.add_spikes(node_ids=0, timestamps=np.random.uniform(0.01, 1000.0, size=10), population='PFC')

Finally we save our spikes to an hdf5 file in the specified SONATA format. Use the *sort_order* attribute to sort the spikes by time (*sort_order.by_time*) or by node_id (*sort_order.by_id*). If *sort_order* is left blank or set to *sort_order.unknown* the spikes will be saved in the same order they were inserted. 

In [6]:
st_buffer.to_sonata('output/recorded_spiketimes.h5', sort_order=sort_order.by_time)

# Poission distributed spike reports

pySonata also includes special tools for creating spike-train reports using a predefine distributions. Below we create a sonata file that creates a population of 100 nodes, each firing with 

In [7]:
psg = PoissonSpikeGenerator()
psg.add(node_ids=range(0, 100), firing_rate=15.0, times=(0.0, 3.0), population='const')


We can also create a non-homogeneous poission distribution by passing in a list of rates (make sure that the rates are always non-negative), here we generate a second population where the nodes vary across the recorded time

In [8]:
times = np.linspace(0, 3.0, 1000)
rates = 15.0 + 15.0*np.sin(times)

psg.add(node_ids=range(0, 100), firing_rate=rates, times=times, population='sinosodial')

In [9]:
psg.to_sonata('output/poisson_spikes.h5')

## Extra Options

### Working with single population

More often then not there is only a single node-population that is being recorded from and having to specify the *population* becomes burdensome. Use the *default_population* parameter so that whenever calling add_spike(s) methods you can ignore the population.

```python
st_buffer = SpikeTrains(default_population='VTA')
st_buffer.add_spike(node_id=0, timestamp=1.0)
...
```




### Excessively large spike trains

pySonata tries to be as memory efficent as it can, however sometimes during a very active simulation with millions of cells there are too many spikes to save in memory. For large simulations ran on machines with limited memory use the *cache_dir* parameter to have spikes temporarly saved to disk preventing out-of-memory errors (but expected a slow-down).

```python
st_buffer = SpikeTrains(cache_dir='output/tmp_spikes')
...
```

### Parallelized simulations

If MPI is installed, you can split the creation of the spike-reports file across different nodes on a multi-core cluster. Initializing and saving the spikes are the same as above, and only the calls to add_spike(s) are distributed among the different cores:

```python
from mpi4py import MPI
rank = MPI.COMM_WORLD.comm.Get_rank()

st_buffer = SpikeTrains(default_population='hpc')
if rank == 0:
    st_buffer.add_spikes(node_ids=0, timestamps=np.linspace(0, 1.0, 20))
else:
    st_buffer.add_spikes(node_ids=rank, timestamps=np.random.uniform(1.0, 2.0, size=20))

st_buffer.to_sontat('hpc_spikes.h5')
```

# Reading Sonata Reports

Now that we have a SONATA file containing spike-trains we can read it in using the **from_sonata** method and use the **populations** property to see what node populations exists in the file

In [10]:
spikes = SpikeTrains.from_sonata('output/recorded_spiketimes.h5')
print(spikes.populations)

['PFC', 'VTA']


In [11]:
# Get the node_ids associated with the 'VTA' population
spikes.nodes('VTA')

[('VTA', 0), ('VTA', 1), ('VTA', 2), ('VTA', 3), ('VTA', 4), ('VTA', 5)]

In [12]:
# find the first and last spike time
spikes.time_range('VTA')

(1.0, 998.577274718506)

In [13]:
# get number of spikes generated by VTA neurons
spikes.n_spikes('VTA')

113

There are a couple of ways to fetch the spikes in the file.

In [14]:
# Returns all spikes as a dataframe
spikes.to_dataframe()

,timestamps,population,node_ids
0,10.223972,PFC,0
1,31.314021,PFC,0
2,321.767321,PFC,0
3,519.829548,PFC,0
4,527.461104,PFC,0
...,...,...,...
108,951.224733,VTA,1
109,959.576032,VTA,2
110,990.913587,VTA,5
111,996.774510,VTA,3


In [15]:
# get the spikes on for VTA node 0.
spikes.get_times(node_id=0, population='VTA')

array([ 1. , 23. , 50.1])

In [16]:
# spikes is a generator method usefull for analyzing very large spike files.
for time, pop, node_id in spikes.spikes(populations='PFC'):
    print(time, pop, node_id)

10.223972437418363 PFC 0
31.314020820945807 PFC 0
321.76732088069923 PFC 0
519.829548301087 PFC 0
527.4611037022787 PFC 0
539.3758044306155 PFC 0
597.786106804225 PFC 0
726.2885222706249 PFC 0
729.1906360853453 PFC 0
860.6247226603169 PFC 0
